# Computational Finance

## Lab Assignment 2

This notebook makes use of a python 3 environment by default.

Authors:
- Kevin de Vries
- Jedda Boyle
- Krishnakanth Sasi

Student numbers:
- 10579869
- 11398221
- 11391952

# Part 1: Basic Option Valuation

# Part 2: Estimation of Sensitivities in MC

# Part 3: Variance Reduction